In [27]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [28]:
# default 

In [29]:
from os import path,listdir
from shutil import rmtree
import pandas as pd
import numpy as np

job_offer_classifier.__version__

In [30]:
# Shutdown TF info logging
import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

## Preprocess the Mocks

In [31]:
data_dir = '../data/raw/mock_email_sentiments/'

### Ivan

In [32]:
IVAN_DIR = path.join(data_dir, 'ivan')
assert path.isdir(IVAN_DIR)

In [33]:
gen_rows = lambda direc, sentiment: (
    {
        'payload': open(path.join(direc, file)).read(),
        'sentiment': sentiment,
        'mocker': 'Ivan'
    } for file in listdir(direc) if file[0] != '.'
)

df_iv = pd.concat(
    pd.DataFrame(gen_rows(path.join(IVAN_DIR, sent), sent))
    for sent in ('negative', 'positive', 'neutral')
).sample(frac=1.0)

In [34]:
df_iv.shape

(30, 3)

In [35]:
for k,a in df_iv.sample(5).payload.iteritems():
    print(k,a,sep='\n'+10*'*'+'\n')

0
**********
Greetings,

I saw the description for the job you sent me and I think it could be
a good change for me. It's a bit different from my usual roles but
I'm certainly up to the challenge. 

Regards. 

6
**********
Hi,

The position matches my profile and the location 
is ideal. Unfortunately it does not meet my current
financial requirements. If the salary is not negotiable 
then I cannot accept. 
9
**********
Greetings,

The job position would be great for me if only I could choose my own work location.
Due to personal reasons I currently cannot be away too ofter or for too long. 

Does the job offer remote work options?
Is it possible to take sick days/personal days?

Regards


8
**********
Greetings,

The job seems to fit my profile but compared to my current
employment it looks like more of the same for me. I'm not interested
in a lateral move but if anything should come up in the future be
sure to keep me in mind.
0
**********
Hi,

Regarding the job opening, I don't think

In [36]:
ex = df_iv.sample().iloc[0]
print(ex.payload.strip(), ex.sentiment, sep='\n\nSentiment: ')

hello,

The job description you sent seems interesting. If the offer still stands
I would like to interview for it.

Sentiment: positive


### Sol

In [37]:
SOL_DIR = path.join(data_dir,'sol')
assert path.isdir(SOL_DIR)

In [38]:
sep_rule = {'negative': 109 * '-', 'neutral': 109 * '-', 'positive': 109 * '-'}
gen_rows = lambda sentiment: (
    {
        'payload': text,
        'sentiment': sentiment,
        'mocker': 'Sol'
    } for text in open(path.join(SOL_DIR, sentiment + '.txt')).read().
    split(sep_rule[sentiment])
)

df_so = pd.concat(
    pd.DataFrame(gen_rows(sent))
    for sent in ('neutral', 'negative', 'positive')
).sample(frac=1.0)

In [39]:
df_so.shape

(30, 3)

In [40]:
for k,a in df_so.sample(5).payload.iteritems():
    print(k,a,sep='\n'+10*'*'+'\n')

1
**********
 

I am available for working only from home, if the job position is flexible with this I am happy to be considered for an interview.

Mrs, House


9
**********
 
I have just enrolled in a PhD program and can not apply for this job.
I will be glad to work with you in the future and look forward to meet with you in another ocassion.

Gabriela Romero

1
**********
 

thank you for considering me for this position,  I am sorry to inform you that I have taken a job in south lake city and will be moving next week.


7
**********
 
Dear recruiter

thank you for getting in touch with me. I am interested in this job position. But  I haven't finished my bachelor's program and can not meet the arrival day. 

Many thanks,
S.

6
**********
 

to whom it may concern,

I am interested and would like to get an interview for this job as soon as possible. BU first I want to know more about the benefits the job offers. 

Mr. P




### Salvador

In [41]:
SALVADOR_FILE = '../data/raw/mock_email_sentiments/salvador.txt'
assert path.isfile(SALVADOR_FILE)

In [42]:
label_map = {
    'considering': 'neutral',
    'acceptance': 'positive',
    'decline': 'negative'
}


def proc_text(text):
    init = text.lower().find('label:')
    end = text[init:].find('\n')
    return [(text[end+2:].strip(), text[init + len('label:'):end+2].strip())]

gen_rows = lambda: (
    {
        'payload': payload,
        'sentiment': label_map[label.lower()],
        'mocker': 'Salvador'
    } for text in open(SALVADOR_FILE).read().split(32 * '-')
    for payload,label in proc_text(text)
)

df_sa = pd.DataFrame(gen_rows()).sample(frac=1.0)

In [43]:
df_sa.shape

(12, 3)

In [44]:
for k,a in df_sa.sample(5).payload.iteritems():
    print(k,a,sep='\n'+10*'*'+'\n')

7
**********
Dear Mr. Jones,

I sincerely appreciate your consideration for the account manager position with your firm. I regret to inform you that I must withdraw my application for the job. My wife has received an attractive promotion with her company that will require relocation to another state, and we will be moving at the end of the summer.

Thank you for the time you spent reviewing my qualifications and meeting with me.

Best regards,
Shaun Murphy
555-862-15756
shaun_mur@mymail.com
6
**********
Dear Randy Johnson:

I very much appreciate your consideration for the DevOps Engineer with Zalando. After further thought, I have decided to withdraw my application for the position.

It was a pleasure meeting you. I appreciate the time you spent discussing the opportunity with me, as well as the information you shared on the job and the company.

Thank you for your time and consideration. I wish you success in finding the perfect candidate to fill this position.

Best regards,
David O

In [45]:
ex = df_sa.sample().iloc[0]
print(ex.payload.strip(), ex.sentiment, sep='\n\nSentiment: ')

Dear Mr. Andrews,

Accept this email as my formal acceptance of the offered position as Cloud Architect. I thank you for the opportunity, and I look forward to applying my talents to the position.

As the offer letter displays and as we previously discussed on the phone, I accept the starting salary of $68,000 with in-lieu work after three months.

I will be meeting the team on July 1, 2020. If you require any additional information from me before then, please let me know. You can reach me at (556) 896-6302 or better by email at sjackson@cloudservices.com.

Thank you again for allowing me to demonstrate my value.
Sincerely,
Samuel L. Jackson

Sentiment: positive


### Join Mocks

In [46]:
df = pd.concat([df_iv,df_so,df_sa],ignore_index=True)

In [47]:
df.shape

(72, 3)

In [48]:
df.sample(2)

,payload,sentiment,mocker
15,"Hello,\n\nI appreciate the consideration you h...",negative,Ivan
18,"Hello,\n\nI'm not interested in this position ...",negative,Ivan


Convert 'positive' and 'negative' values to 1 and 0 values, respectively.

In [49]:
df.sentiment = df.sentiment.apply(lambda s: int(s=='positive'))

Save dataframe in a file the positive and the negative cases, and the neutral case in another file

In [50]:
df[df.sentiment != 'neutral'].to_csv('../data/interim/mocks_pos_neg.csv',index=False)
df[df.sentiment == 'neutral'].to_csv('../data/interim/mocks_neutral.csv',index=False)
#assert np.all((df.sentiment !='neutral').values)

/opt/conda/lib/python3.7/site-packages/pandas/core/ops/array_ops.py:253: FutureWarning: elementwise comparison failed; returning scalar instead, but in the future will perform elementwise comparison
  res_values = method(rvalues)


In [51]:
ex = df.sample().iloc[0]
print(ex.payload)
print(f'Sentiment:\n{ex.sentiment}')

 
Dear recruiter

thank you for getting in touch with me. I am interested in this job position. But  I haven't finished my bachelor's program and can not meet the arrival day. 

Many thanks,
S.

Sentiment:
0


## Enrique